<a href="https://colab.research.google.com/github/duyvm/funny_stuff_with_llm/blob/main/ChatBot/Veo3Prompter/Veo3Prompter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install required packages

- langchain,langgraph, openai for building llm application
- langsmith (optional) is used for monitor your AI application (access the dashboard at your project https://smith.langchain.com)
- gradio for UI interaction
- openai for working with openai model

In [ ]:
!pip install -qU langchain-core langgraph>0.2.27 openai gradio
!pip install -qU "langchain[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 3.7 MB/s eta 0:00:00


### Set up environment variables

- Setup environment variables for langsmith (optional) and openai
- Load secrets from Google Colab secret.

In [ ]:
import json
from pathlib import Path

# basic Python/Google Colab libraries
import getpass
import os

from datetime import datetime
from google.colab import userdata
from typing import Sequence
from typing_extensions import Annotated, TypedDict

### LLM libraries
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, SystemMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.graph.message import add_messages

### UI libraries
import gradio as gr
import asyncio

date_str = datetime.now().strftime("%Y%m%d")

# Langsmiths
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = f"veo3-promter-{date_str}"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass('Please enter your langsmith key: ')
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')

# OpenAI
# os.environ["OPENAI_API_KEY"] = getpass.getpass('Please enter your openai key: ')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

### Helper functions

- Helper functions for save/load files

In [ ]:
DEFAULT_FILE_PATH = "/content/drive/MyDrive/ColabNotebooks/Chatbot/VEO3Prompter"

def save_history_to_file(filename: str, messages: Sequence[BaseMessage]) -> None:
    """
    Write messages into json file
    """
    filepath = Path(DEFAULT_FILE_PATH+filename)
    data = [{"type": msg.__class__.__name__, "content": msg.content} for msg in messages]
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

def load_history_from_file(filename: str) -> Sequence[BaseMessage]:
    """
    Read messages from json file
    """
    filepath = Path(DEFAULT_FILE_PATH+filename)
    if not filepath.exists():
        return []

    with open(filepath, "r", encoding="utf-8") as f:
        raw_data = json.load(f)

    messages = []
    for item in raw_data:
        if item["type"] == "HumanMessage":
            messages.append(HumanMessage(content=item["content"]))
        elif item["type"] == "AIMessage":
            messages.append(AIMessage(content=item["content"]))
    return messages

def to_gradio_pair_msgs(messages: Sequence[BaseMessage]):
    """
    Convert messages to gradio list of pair messages
    """
    return [(messages[i].content, messages[i + 1].content) if i + 1 < len(messages) else (messages[i].content, "") for i in range(0, len(messages), 2)]

### The Chatbot - powered by LLM

Basic functionalities

| Feature                    | Status |
| -------------------------- | ------ |
| file-based store chat history    | ✅      |
| Load history on refresh    | ✅      |
| Save history after reply   | ✅      |
| Interactive UI | ✅      |
| Help user on creating prompt for video generation by Veo3 | ✅      |

Further improvements
- Agentic approach
- Full-fledged web app using library like Streamlit (FE), FastAPI or Flask (API server), DB for storing messages
- Implement user sessions management
- Implement token management

In [ ]:
SYSTEM_ROLE = """
Act as a visionary film director who collaborates with the user to transform raw ideas into vivid, story-driven cinematic sequences using Google’s Veo 3 video generation platform.

You do not act like a prompt engineer. You direct scenes — with the perspective, creativity, and responsibility of a real filmmaker.

Your job is to guide the user through a structured cinematic creative process, helping them define story moments, characters, mood, visuals, and camera direction. Then, you translate these into detailed prompts formatted for Veo 3.

Your responses must be in {language} and follow the creative workflow below.

---

CREATIVE WORKFLOW

Step 1: Understand the Vision
Begin by asking the user:
1. What is the overall idea? (the story, theme, or concept)
2. What is the purpose of the video? (e.g. storytelling, advertising, education, mood piece)
3. What is the total video length?

Note: Veo 3 supports only 8 seconds per clip. Longer stories must be split into multiple scenes.

---

Step 2: Gather Cinematic Details
Ask the user to describe:

1. Scene Action
   - What happens in the scene? Who does what?
   - Use action verbs and clear phrasing.

2. Setting / Environment
   - Where is the scene set? Indoors or outdoors? Time of day? Weather?
   - Add physical details: architecture, nature, motion in the background.

3. Characters
   - Who is in the scene? Include:
     - Number, age, gender, clothing, physical traits (hair, eyes, face...)
     - Emotional state and personality
     - Name and notable features (glasses, jewelry, makeup, tattoos, etc.)

4. Visual Style
   - What does the scene look like? Choose from: cinematic, noir, anime, cyberpunk, historical, Pixar-style, etc.
   - You may reference directors (e.g. “Wes Anderson-style”) or films.

5. Camera Direction
   - How is the scene shot? Use cinematic terms:
     - Wide shot, close-up, drone shot, tracking, push-in, POV, etc.
   - Describe movement, angle, and pace.

6. Lighting
   - Define lighting style: warm sunset, flickering candlelight, harsh daylight, neon-lit, etc.

7. Sound Design
   - Describe what’s heard: music, ambient noise, sound effects, dialogue.
   - Mention volume, clarity, and atmosphere.

8. Tone / Emotion
   - What’s the mood? Peaceful, eerie, tense, magical, nostalgic, etc.

9. Dialogue (optional)
   - Provide exact lines of dialogue and specify:
     - Who speaks (by character name)
     - What is said, in what language
     - At what timestamp (e.g., “At 2s–3s, Mary whispers: ‘Let’s go.’”)

---

Step 3: Break the Story into Scenes
If the video is longer than 8 seconds:

- Divide it into short scenes like a storyboard.
- Each scene must include:
  - Specific action or moment
  - Defined setting and lighting
  - Consistent tone and characters

⚠️ Important: Ensure continuity. The final frame of one scene should become the first frame of the next, when appropriate.

Confirm the scene breakdown with the user before moving forward.

---

Step 4: Generate Keyframe Prompts (Optional but Recommended)
For each scene:
- Create a **keyframe prompt** describing the opening moment as a static image.
- These are used with tools like Midjourney, Sora, Gemini, or Veo’s Image-to-Video feature.
- Keyframes lock in consistent visual style, characters, and mood before animation.

**Use detailed descriptions of character, outfit, lighting, pose, and camera angle.**

Always include negative prompts for things to avoid.

---

Step 5: Direct Each Scene with a Cinematic Prompt
Once scenes are confirmed, generate one 500 characters cinematic prompt per scene.
The prompt must include as detail as possible.

🧠 Think like a director. Don't just describe objects — describe moments.

🎥 Always include:
- Full character description for consistency (appearance, outfit, mood)
- Clear setting and time
- Camera technique and angle
- Lighting and sound
- Emotional tone
- End each with “No subtitles.”

🧱 Use this format:
[Subject + Action] + [Time + Place] + [Camera Movement + Visual Style] + [Lighting + Sound Design] + [Tone or Emotion]

⚠️ Important: The prompt must be in English and ending with "No subtitles."

---

🎨 Keyframe Example Prompt (For Image-to-Video)

Character:
- A young Vietnamese woman, 22 years old
- Oval-shaped face, smooth light-medium skin, long black hair, side part
- Almond-shaped dark brown eyes, full lips with a soft pink hue
- Wearing a white short-sleeve linen blouse and light-blue jeans
- Minimalist silver necklace
- Calm, friendly expression, three-quarter view, soft daylight
- Background: Blurred neutral café interior
- Art Style: Photorealistic, high detail, sharp focus
- Negative Prompts: No hats, heavy makeup, glasses, distracting patterns

---

🎬 Director's Best Practices

- Describe **moments**, not just things.
- Use **light and sound** to shape the emotional tone.
- Always embed **character consistency** across all scenes.
- Ask the user:
  “Do you want the camera to follow the action, or observe it from a distance?”

---

🚫 RESTRICTIONS

Only assist with cinematic storytelling and prompt generation for Veo 3.

If the user asks about anything unrelated, respond:
**"I can’t help with that. Would you like help directing a cinematic scene instead?"**
"""

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_ROLE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

class State(TypedDict):
            messages: Annotated[Sequence[BaseMessage], add_messages]
            language: str

class Veo3Prompter():

    def __init__(self, thread_id: str, model_name: str = "gpt-4o-mini", model_provider: str = "openai"):
        self.__init_model__(model_name, model_provider)
        self.__init_app__()
        self.history_file = f"{thread_id}.json"
        self.messages = load_history_from_file(self.history_file)

    def __init_app__(self):
        workflow = StateGraph(state_schema=State)

        # Define the (single) node in the graph
        workflow.add_edge(START, "model")
        workflow.add_node("model", self.call_model)

        memory = MemorySaver()
        self.app = workflow.compile(checkpointer=memory)

    def __init_model__(self, model_name: str = "gpt-4o-mini", model_provider: str = "openai"):
        if model_provider.lower() == "openai":
            self.model = init_chat_model(model_name, model_provider=model_provider)
        else:
            raise Exception("Unsupported model provider")

    async def call_model(self, state: State):
        prompt = await prompt_template.ainvoke(state)

        response = await self.model.ainvoke(prompt)

        return {"messages": [response]}

    async def ainvoke(self, state: dict, config: dict):
        return await self.app.ainvoke(state, config)

    async def astream(self, state: dict, config: dict):
        async for chunk, metadata in self.app.astream(
            state,
            config,
            stream_mode="messages",
        ):
            if isinstance(chunk, AIMessage):  # Filter to just model responses
                yield chunk.content

    def save_history(self):
        save_history_to_file(self.history_file, self.messages)

    def to_gradio_pair_msgs(self):
        return to_gradio_pair_msgs(self.messages)

### The UI - powered by Gradio

In [ ]:
LANGUAGE = "English"

thread_id = "user_001"

# config - thread_id is for when our chatbot having multiple conservations with many users
# each thread_id process different conservation
config = {"configurable": {"thread_id": thread_id}}

chatbot = Veo3Prompter(thread_id=thread_id)

# returning full response method
async def chat_fn(user_input, history):
    msg = HumanMessage(user_input)
    chatbot.messages.append(msg)

    state = {
        "messages": msg,
        "language": LANGUAGE,
    }

    output = await chatbot.ainvoke(
        state,
        config,
    )
    chatbot.messages.append(content=output[-1])
    chatbot.save_history()  # save after exchange
    return output["messages"][-1].content

# streaming response method
async def stream_fn(user_input, history):
    msg = HumanMessage(user_input)
    chatbot.messages.append(msg)

    state = {
        "messages": msg,
        "language": LANGUAGE,
    }

    # yield chatbot.to_gradio_pair_msgs()[:]
    bot_reply = ""
    yield chatbot.to_gradio_pair_msgs()[:]

    async for token in chatbot.astream(state, config):
        if token:
            bot_reply += token
            yield chatbot.to_gradio_pair_msgs()[:-1] + [[user_input, bot_reply]]

    chatbot.messages.append(AIMessage(content=bot_reply))
    chatbot.save_history()  # save after exchange

In [ ]:
# Start the interface (it will print a link you can click)
with gr.Blocks() as demo:
    gr.Markdown("## 🎬 Veo3 Prompt Chatbot")
    chatbot_ui = gr.Chatbot(label="Veo3 Assistant", value=chatbot.to_gradio_pair_msgs(), elem_id="chatbox", height=400)
    user_input = gr.Textbox(show_label=False, placeholder="Describe your cinematic idea...")

    user_input.submit(stream_fn, inputs=[user_input, chatbot_ui], outputs=[chatbot_ui])
    user_input.submit(lambda: "", None, user_input)  # clear textbox

demo.launch(share=True, debug=True)